In [1]:
import ee
import geemap
import math

In [2]:
Map = geemap.Map(center=(44.79, 77.92), zoom=7)
Map.add_basemap('Esri.WorldImagery')

In [3]:
# Loading datasets
CHIRPS = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")
soil = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")
DEM = ee.Image("USGS/SRTMGL1_003")
sentinel2 = ee.ImageCollection("COPERNICUS/S2")
modis = ee.ImageCollection("MODIS/061/MCD12Q1")

In [4]:
# Region of Interest
path_shapefile = "data/Alm_Zh_regions.shp"
ROI = geemap.shp_to_ee(path_shapefile)
style = {"color": "0000ffff", "width": 0.5, "lineType": "solid", "fillColor": "00000080"}

# Date
start_date = '2023-01-01'
end_date = '2023-12-01'

# adding to the map
Map.addLayer(ROI.style(**style), {}, 'Region of Interest')

In [14]:
Map

Map(bottom=12114.0, center=[44.79, 77.92], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [6]:
# R Factor

current = CHIRPS.filter(ee.Filter.date(start_date, end_date)).sum().clip(ROI)
Map.addLayer(current, {}, "Annual Rain", 0)
R = ee.Image(current.multiply(0.363).add(79)).rename("R")
Map.addLayer(
    R,
    {
        "min": 300,
        "max": 900,
        "palette": ["a52508", "ff3818", "fbff18", "25cdff", "2f35ff", "0b2dab"],
    },
    "R Factor Map",
    0,
)

In [7]:
# K Factor

soil = soil.select("b0").clip(ROI).rename("soil")
Map.addLayer(
    soil,
    {
        "min": 0,
        "max": 100,
        "palette": ["a52508", "ff3818", "fbff18", "25cdff", "2f35ff", "0b2dab"],
    },
    "Soil",
    0,
)

K = (
    soil.expression(
        "(b('soil') > 11) ? 0.0053"
        + ": (b('soil') > 10) ? 0.0170"
        + ": (b('soil') > 9) ? 0.045"
        + ": (b('soil') > 8) ? 0.050"
        + ": (b('soil') > 7) ? 0.0499"
        + ": (b('soil') > 6) ? 0.0394"
        + ": (b('soil') > 5) ? 0.0264"
        + ": (b('soil') > 4) ? 0.0423"
        + ": (b('soil') > 3) ? 0.0394"
        + ": (b('soil') > 2) ? 0.036"
        + ": (b('soil') > 1) ? 0.0341"
        + ": (b('soil') > 0) ? 0.0288"
        + ": 0"
    )
    .rename("K")
    .clip(ROI)
)

Map.addLayer(
    K,
    {
        "min": 0,
        "max": 0.06,
        "palette": ["a52508", "ff3818", "fbff18", "25cdff", "2f35ff", "0b2dab"],
    },
    "K Factor Map",
    0,
)

In [8]:
# LS Factor

elevation = DEM.select("elevation")
slope1 = ee.Terrain.slope(elevation).clip(ROI)
# Converting Slope from Degrees to %
slope = slope1.divide(180).multiply(math.pi).tan().multiply(100)
Map.addLayer(
    slope,
    {
        "min": 0,
        "max": 15,
        "palette": ["a52508", "ff3818", "fbff18", "25cdff", "2f35ff", "0b2dab"],
    },
    "slope in %",
    0,
)

LS4 = math.sqrt(500 / 100)
LS3 = ee.Image(slope.multiply(0.53))
LS2 = ee.Image(slope).multiply(ee.Image(slope).multiply(0.076))
LS1 = ee.Image(LS3).add(LS2).add(0.76)
LS = ee.Image(LS1).multiply(LS4).rename("LS")

Map.addLayer(
    LS,
    {
        "min": 0,
        "max": 90,
        "palette": ["a52508", "ff3818", "fbff18", "25cdff", "2f35ff", "0b2dab"],
    },
    "LS Factor Map",
    0,
)

In [9]:
# C Factor
# current = CHIRPS.filter(ee.Filter.date(start_date, end_date)).sum().clip(ROI)

s2 = sentinel2.filter(ee.Filter.date(start_date, end_date)).median().clip(ROI)
image_ndvi = s2.normalizedDifference(["B8", "B4"]).rename("NDVI")

Map.addLayer(
    image_ndvi,
    {
        "min": 0,
        "max": 0.85,
        "palette": [
            "FFFFFF",
            "CC9966",
            "CC9900",
            "996600",
            "33CC00",
            "009900",
            "006600",
            "000000",
        ],
    },
    "NDVI",
    0,
)

alpha = ee.Number(-2)
beta = ee.Number(1)

C1 = image_ndvi.multiply(alpha)
oneImage = ee.Image(1).clip(ROI)
C2 = oneImage.subtract(image_ndvi)
C3 = C1.divide(C2).rename("C3")
C4 = C3.exp()

maxC4 = C4.reduceRegion(
        geometry = ROI,
        reducer = ee.Reducer.max(),
        scale = 3000,
        maxPixels = 475160679
)

C5 = maxC4.toImage().clip(ROI)
minC4 = C4.reduceRegion(
        geometry = ROI.geometry().getInfo(),
        reducer = ee.Reducer.min(),
        scale = 3000,
        maxPixels = 475160679,
)

C6 = minC4.toImage().clip(ROI)
C7 = C4.subtract(C6)
C8 = C5.subtract(C6)

C = C7.divide(C8).rename("C")

Map.addLayer(
    C,
    {
        "min": 0,
        "max": 1,
        "palette": [
            "FFFFFF",
            "CC9966",
            "CC9900",
            "996600",
            "33CC00",
            "009900",
            "006600",
            "000000",
        ],
    },
    "C Map",
    0,
)

In [10]:
# P Factor

lulc = (
    modis.filterDate('2021-01-01', '2022-01-01').select("LC_Type1").first().clip(ROI).rename("lulc")
)
Map.addLayer(lulc, {}, "lulc", 0)

# Combined LULC & slope in single image
lulc_slope = lulc.addBands(slope)

# Create P Facor map using an expression
P = (
    lulc_slope.expression(
        "(b('lulc') < 11) ? 0.8"
        + ": (b('lulc') == 11) ? 1"
        + ": (b('lulc') == 13) ? 1"
        + ": (b('lulc') > 14) ? 1"
        + ": (b('slope') < 2) and((b('lulc')==12) or (b('lulc')==14)) ? 0.6"
        + ": (b('slope') < 5) and((b('lulc')==12) or (b('lulc')==14)) ? 0.5"
        + ": (b('slope') < 8) and((b('lulc')==12) or (b('lulc')==14)) ? 0.5"
        + ": (b('slope') < 12) and((b('lulc')==12) or (b('lulc')==14)) ? 0.6"
        + ": (b('slope') < 16) and((b('lulc')==12) or (b('lulc')==14)) ? 0.7"
        + ": (b('slope') < 20) and((b('lulc')==12) or (b('lulc')==14)) ? 0.8"
        + ": (b('slope') > 20) and((b('lulc')==12) or (b('lulc')==14)) ? 0.9"
        + ": 1"
    )
    .rename("P")
    .clip(ROI)
)
Map.addLayer(P, {}, "P Factor", 0)

In [12]:
# Estimating soil loss

soil_loss = ee.Image(R.multiply(K).multiply(LS).multiply(C).multiply(P)).rename(
    "Soil Loss"
)

style = ["490eff", "12f4ff", "12ff50", "e5ff12", "ff4812"]

Map.addLayer(soil_loss, {"min": 0, "max": 10, "palette": style}, "Soil Loss", 0)

SL_class = (
    soil_loss.expression(
        "(b('Soil Loss') < 5) ? 1"
        + ": (b('Soil Loss') < 10) ? 2"
        + ": (b('Soil Loss') < 20) ? 3"
        + ": (b('Soil Loss') < 40) ? 4"
        + ": 5"
    )
    .rename("SL_class")
    .clip(ROI)
)
Map.addLayer(SL_class, {"min": 0, "max": 5, "palette": style}, "Soil Loss Class")

SL_mean = soil_loss.reduceRegion(
    geometry = ROI,
    reducer = ee.Reducer.mean(),
    scale =  500,
    maxPixels = 475160679,
)

print("Mean Soil Loss", SL_mean.get("Soil Loss"))

# Add reducer output to the Features in the collection.
maineMeansFeatures = soil_loss.reduceRegions(
        collection = ROI,
        reducer = ee.Reducer.mean(),
        scale = 500,

)

print("Mean Soil Loss of Each Subbasins", maineMeansFeatures)

# Calculating Area
areaImage = ee.Image.pixelArea().addBands(SL_class)
areas = areaImage.reduceRegion(
        reducer = ee.Reducer.sum().group(
                groupField = 1,
                groupName = "class",
        ),
        geometry = ROI.geometry(),
        scale = 500,
        maxPixels = 1e10,
)

# print(areas)

classAreas = ee.List(areas.get("groups"))


def func_zhd(item):
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get("class")).format()
    return ee.List(classNumber)


className = classAreas.map(func_zhd)

# print(className)


def func_qfd(item):
    areaDict = ee.Dictionary(item)
    area = ee.Number(areaDict.get("sum")).divide(1e6).round()
    return ee.List(area)


Area = classAreas.map(func_qfd)

# print(Area)
className2 = ee.List(
    [
        "Slight (<10)",
        "Moderate (10-20)",
        "High (20-30)",
        "Very high (30-40)",
        "Severe (>40)",
    ]
)

# print(
#     ui.Chart.array.values(Area, 0, className2)
#     .setChartType("PieChart")
#     .setOptions(
#         {
#             pointSize: 2,
#             title: "Soil Loss",
#         }
#     )
# )

###############
def calculateClassArea(feature):
    areas = (
        ee.Image.pixelArea()
        .addBands(SL_class)
        .reduceRegion(
                reducer = ee.Reducer.sum().group(
                        groupField = 1,
                        groupName = "class",
                ),
                geometry = feature.geometry(),
                scale = 500,
                maxPixels = 1e10,
        )
    )

    classAreas = ee.List(areas.get("groups"))


    def func_bkl(item):
        areaDict = ee.Dictionary(item)
        classNumber = ee.Number(areaDict.get("class")).format()
        area = ee.Number(areaDict.get("sum")).round()
        return ee.List([classNumber, area])


    classAreaLists = classAreas.map(func_bkl)

    result = ee.Dictionary(classAreaLists.flatten())
    district = feature.get("HYBAS_ID")
    return ee.Feature(feature.geometry(), result.set("district", district))

districtAreas = ROI.map(calculateClassArea)
print (districtAreas)

classes = ee.List.sequence(1, 5)
outputFields = ee.List(["district"]).cat(classes).getInfo()

# geemap.ee_export_image_to_drive(
#     {
#         "collection": districtAreas,
#         "description": "class_area_by_subbasin",
#         "folder": "earth_engine_data",
#         "fileNamePrefix": "RUSLE_soil_loss",
#         "fileFormat": "CSV",
#         "selectors": outputFields,
#     }
# )

Mean Soil Loss 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
legend = ui.Panel({"style": {"position": "bottom-left", "padding": "8px 15px"}})

# Create legend title
legendTitle = ui.Label(
    {
        "value": "Soil Loss (t/hac/year)",
        "style": {
            "fontWeight": "bold",
            "fontSize": "18px",
            "margin": "0 0 4px 0",
            "padding": "0",
        },
    }
)

# Add the title to the panel
legend.add(legendTitle)

# Creates and styles 1 row of the legend.
def makeRow(color, name):

    # Create the label that is actually the colored box.
    colorBox = ui.Label(
        {
            "style": {
                "backgroundColor": "#" + color,
                # Use padding to give the box height and width.
                "padding": "8px",
                "margin": "0 0 4px 0",
            }
        }
    )

    # Create the label filled with the description text.
    description = ui.Label({"value": name, "style": {"margin": "0 0 4px 6px"}})

    # return the panel
    return ui.Panel(
        {
            "widgets": [colorBox, description],
            "layout": ui.Panel.Layout.Flow("horizontal"),
        }
    )


#  Palette with the colors
palette = style

# name of the legend
# names = ["Slight","Moderate","High","Very high","Severe"]
names = [
    "Slight (<10)",
    "Moderate (10-20)",
    "High (20-30)",
    "Very high (30-40)",
    "Severe (>40)",
]

# Add color and and names
for i in range(0, 5, 1):
    legend.add(makeRow(palette[i], names[i]))

# add legend to map (alternatively you can also print the legend to the console)
Map.add(legend)
Map

NameError: name 'ui' is not defined

source: https://github.com/sukantjain/Google-Earth-Engine/blob/main/JavaScript/RUSLE%20in%20GEE.js